# Intro to model management, importing and searching for data
## Install libs

In [ ]:
import sys
!{sys.executable} -m pip install requests

## Basic Variables
### Credentials
If you aren't authentication with auth0, you need to use our simple refresh/access token approach. This requires a username and password to login

In [ ]:
import getpass
from pathlib import Path

default_username = 'demo.user'
#username = input(f"Enter your username [{default_username}]: ") or default_username

username = default_username
password_file = Path('/Users/paul/.cyoda/demo.passwd')

if password_file.exists():
    # Your action here, e.g., read the file, etc.
    with password_file.open('r') as file:
        password = file.read().rstrip()
else:
    # Prompt for credentials when the notebook runs
    password = getpass.getpass("Enter your password: ")

credentials = {
    'username': username,
    'password': password
}

### Parameters and Endpoints

In [ ]:
#namespace = 'put your cyoda namespace here'
# api_url = f"https://{namespace}.cyoda.net/api"

# As a Cyoda developer, I can run the service on my laptop. :-)
api_url = 'http://localhost:8082/api'

# Well be using this model name and version
model_name = 'prizes'
model_version = 1

# Dummy values, to be set later
entity_id=''
update_transition='UPDATE'

login_endpoint = f"{api_url}/auth/login"
token_endpoint = f"{api_url}/auth/token"

## Some access token functions


In [ ]:
import requests
import json
import sys

def login_and_get_refresh_token(credentials):

    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'Content-Type': 'application/json'
    }
    
    payload = json.dumps(credentials)
    
    response = requests.post(login_endpoint, headers=headers, data=payload)

    if response.status_code == 200:
        # Assuming the refresh token is returned in the 'refresh_token' field
        refresh_token = response.json().get('refreshToken')
        return refresh_token
    else:
        raise Exception(f"Login failed: {response.status_code} {response.text}")
    

def get_access_token(refresh_token):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {refresh_token}'
    }
    response = requests.get(token_endpoint, headers=headers)

    if response.status_code == 200:
        token_data = response.json()
        access_token = token_data.get('token')
        #token_expiry = token_data.get('tokenExpiry')
        return access_token
    else:
        raise Exception(f"Token refresh failed: {response.status_code} {response.text}")



In [ ]:



def model_exists(model_name,model_version):
    export_model_url = f"{api_url}/treeNode/model/export/SIMPLE_VIEW/{model_name}/{model_version}"
    
    response = requests.get(export_model_url, headers=headers)
    
    if response.status_code == 200:
        return True
    elif response.status_code == 404:
        return False
    else:
        raise Exception(f"Get: {response.status_code} {response.text}")



def get_model(model_name,model_version):
    export_model_url = f"{api_url}/treeNode/model/export/SIMPLE_VIEW/{model_name}/{model_version}"
    
    response = requests.get(export_model_url,headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Getting the model failed: {response.status_code} {response.text}")



def get_model_state(model_name,model_version):
    export_model_url = f"{api_url}/treeNode/model/export/SIMPLE_VIEW/{model_name}/{model_version}"
    
    response = requests.get(export_model_url,headers=headers)
    
    if response.status_code == 200:
        return response.json().get('currentState')            
    else:
        raise Exception(f"Failed to get the model: {response.status_code} {response.text}")  



def unlock_model(model_name,model_version):
    unlock_model_url = f"{api_url}/treeNode/model/{model_name}/{model_version}/unlock"

    response = requests.put(unlock_model_url,headers=headers)
    if response.status_code == 200:
        print('model unlocked')
    else:
        raise Exception(f"Unlock failed: {response.status_code} {response.text}")



def lock_model(model_name,model_version):
    lock_model_url = f"{api_url}/treeNode/model/{model_name}/{model_version}/lock"

    response = requests.put(lock_model_url,headers=headers)
    if response.status_code == 200:
        print('model locked')
    else:
        raise Exception(f"Lock failed: {response.status_code} {response.text}")



def delete_model(model_name,model_version):
    model_url               = f"{api_url}/treeNode/model/{model_name}/{model_version}"
    
    response = requests.delete(model_url,headers=headers)
    if response.status_code == 200:
        print('model deleted')
    else:
        raise Exception(f"Deletion of the model failed: {response.status_code} {response.text}")



def delete_all_entities(model_name,model_version):
    delete_entities_url = f"{api_url}/entity/TREE/{model_name}/{model_version}"

    response = requests.delete(delete_entities_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Deletion failed: {response.status_code} {response.text}")


def derive_model_from_sample_data(model_name,model_version,payload):
    import_model_url = f"{api_url}/treeNode/model/import/JSON/SAMPLE_DATA/{model_name}/{model_version}"
    response = requests.post(import_model_url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Save failed: {response.status_code} {response.text}")

def create_entity(model_name,model_version,json_payload):
    create_entity_url = f"{api_url}/entity/JSON/TREE/{model_name}/{model_version}"

    params = {
        'transactionTimeoutMillis': '10000'
    }

    response = requests.post(create_entity_url, headers=headers, params=params, data=json_payload)

    if response.status_code == 200:
        json_response = response.json()
        print(response.text)
        
        # Assert that there is only one transaction
        assert len(json_response) == 1, f"Expected 1 transaction, but got {len(json_response)}"
        
        transaction = json_response[0]
        entity_ids = transaction['entityIds']
        
        # Assert that the list of entityIds has exactly one element
        assert len(entity_ids) == 1, f"Expected 1 entityId, but got {len(entity_ids)}"
        
        return entity_ids[0]
    else:
        raise Exception(f"Save failed: {response.status_code} {response.text}")

def find_update_transition(entity_id):
    get_transitions_url = f"{api_url}/platform-api/entity/fetch/transitions"

    params = {
        'entityClass': 'com.cyoda.tdb.model.treenode.TreeNodeEntity',
        'entityId': f"{entity_id}"
    }
    
    print(f"Getting possible transitions for {entity_id})")
    
    response = requests.get(get_transitions_url, headers=headers, params=params)
    
    if response.status_code == 200:
        json_response = response.json()
        
        print(json.dumps(json_response, indent=4))
        
        # Find the first item that starts with 'update' (ignoring case)
        try:
            update_transition = next(item for item in json_response if item.lower().startswith('update'))
        except StopIteration:
            raise ValueError("No item starting with 'update' was found in the response (case-insensitive).")
    
    else:
        raise Exception(f"Getting transitions failed: {response.status_code} {response.text}")

    return update_transition


def update_entity(entity_id,json_payload):
    update_entity_url = f"{api_url}/entity/JSON/TREE/{entity_id}/{update_transition}"

    params = {
    'transactionTimeoutMillis': '10000'
    }
    response = requests.put(update_entity_url, headers=headers, params=params, data=json_payload)

    if response.status_code == 200:
        json_response = response.json()
        
        entity_ids = json_response['entityIds']
        
        # Assert that the list of entityIds has exactly one element
        assert len(entity_ids) == 1, f"Expected 1 entityId, but got {len(entity_ids)}"
        
        assert entity_id == entity_ids[0]
    else:
        raise Exception(f"Update failed: {response.status_code} {response.text}")

    return json_response


## Login to get a long lived refresh token

In [ ]:
refresh_token = login_and_get_refresh_token(credentials=credentials)

## Get an access token from the refresh token
Although the login will give you an access token, in secure environments, this will be short lived. So it's better to use the refresh token to get an access token each time

In [ ]:
access_token = get_access_token(refresh_token=refresh_token)
headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

## Check if the nobel prize model exists

In [ ]:
model_exists = model_exists(model_name,model_version)
print(f"model_exists = {model_exists}")

## Delete any data present for that model

In [ ]:
import requests
import json

if model_exists:
    print(f"Deleting all data for model {model_name} {model_version}")
    delete_result = delete_all_entities(model_name,model_version)
    print(delete_result)
else:
    print('Nothing to delete. Model does not exist')

## Unlock the model if locked

In [ ]:
import requests
import json

if model_exists:
    current_model_state = get_model_state(model_name,model_version)
    
    if current_model_state == 'LOCKED':
        unlock_model(model_name,model_version)
    else:
        print('Model not locked')

## Delete the model
but only if it exists

In [ ]:
import requests
import json

if model_exists:
    delete_model(model_name,model_version)


## Use sample data to define the model

In [ ]:
import json


# This sample contains the valid structure for the model
file_path = './src/main/resources/cyoda/config/nobel-prizes/sample-data/prize-physics-2019.json'

with open(file_path, 'r') as file:
     file_contents = json.load(file)

payload = json.dumps(file_contents)

model_id = derive_model_from_sample_data(model_name,model_version,payload)

print(f"model id = {model_id}")


## Get the model and print it

In [ ]:
import requests
import json

model_view = get_model(model_name,model_version)
print(json.dumps(model_view, indent=4))


## Lock the model
We have to lock models before we can save data against it

In [ ]:
lock_model(model_name,model_version)

## Save an entity
We will save a partial data set for nobel prizes and then update it with a full set afterwards, to demonstrate also how to update entities.
We'll capture the entity id for later use.

In [ ]:
# This dataset in only a partial list of all prizes
file_path = './src/main/resources/cyoda/config/nobel-prizes/sample-data/prize-2010-2020.json'

with open(file_path, 'r') as file:
    file_contents = json.load(file)
    
json_payload = json.dumps(file_contents)
entity_id = create_entity(model_name,model_version,json_payload)    


## Update an entity
Now we'll update the entity with the full nobel prize dataset.
To update we do a PUT request with the transition name as part of the request path.

Since the workflow is configurable and can be different for different entities and also depends on the entity state, theoreticall, we need to get the state of the transition, and the possible transitions for that entity in that state.

You can get the possible transitions for a given entity in a separate request. We haven't yet provided a CaaS specific endpoint for this, so we have to use the platform endpoint, where you must pass the actual Cyoda POJO entity that physically stores your data.

### Find the update transition

In [ ]:
update_transition = find_update_transition(entity_id)
print(f"Using transition: {update_transition}")

### Do the update with the right transition
Now that we have found the right transition for this entity in its current state, we can launch the update with that transition.

In other words, if you run this twice, the assertions will fail.

In [ ]:
import json
import time

# This dataset in has all prizes until a recent date, I think 2020
file_path = './src/main/resources/cyoda/config/nobel-prizes/sample-data/prize.json'

with open(file_path, 'r') as file:
     file_contents = json.load(file)

json_payload = json.dumps(file_contents)
print(f"launching update on entity {entity_id}...")
json_response = update_entity(entity_id,json_payload)

print(json.dumps(json_response, indent=4))